In [1]:
!pip install gcsfs

In [2]:
%%time
import pandas as pd
import os, sys, time, json, re, string
from google.cloud import storage

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only
from pyspark.sql.types import IntegerType

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer, StopWordsRemover
from pyspark.ml.feature import VectorAssembler, PCA

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

3.6.10
CPU times: user 61.9 ms, sys: 12.2 ms, total: 74.1 ms
Wall time: 71.7 ms


In [3]:
%%time
sc

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


<SparkContext master=yarn appName=PySparkShell>

In [4]:
%%time
spark = SparkSession.builder \
        .appName("fakenews_baseline_model") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "24g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "24g") \
        .getOrCreate()

CPU times: user 6.37 ms, sys: 0 ns, total: 6.37 ms
Wall time: 9.38 ms


In [5]:
%%time
df_fake_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/100k_fake_news_cleaned_dataset.csv')
df_fake_train[['authors_missing']] = df_fake_train[['authors_missing']].astype(int)
df_fake_train['label'] = 1
df_reliable_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/100k_reliable_news_cleaned_dataset.csv')
df_reliable_train[['authors_missing']] = df_reliable_train[['authors_missing']].astype(int)
df_reliable_train['label'] = 0
df_news_train = pd.concat([df_fake_train, df_reliable_train])
df_news_train[df_news_train.columns] = df_news_train[df_news_train.columns].astype(str)
print(df_news_train.info())
df_news_train

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217806 entries, 0 to 108010
Data columns (total 7 columns):
domain             217806 non-null object
type               217806 non-null object
content            217806 non-null object
title              217806 non-null object
authors            217806 non-null object
authors_missing    217806 non-null object
label              217806 non-null object
dtypes: object(7)
memory usage: 13.3+ MB
None
CPU times: user 9.11 s, sys: 1.08 s, total: 10.2 s
Wall time: 20.7 s


,domain,type,content,title,authors,authors_missing,label
0,beforeitsnews.com,fake,Boehner presses to make tax cuts permanent\n\n...,Boehner presses to make tax cuts permanent,United Liberty,0,1
1,beforeitsnews.com,fake,An Armed Good Samaritan Who Doesn’t Want The S...,An Armed Good Samaritan Who Doesn’t Want The S...,The Real Revo,0,1
2,beforeitsnews.com,fake,(Before It's News)\n\nby Rob Morphy\n\nLegends...,Village Of The Dead: The Anjikuni Mystery,"Rob Morphy, Mort Amsel",0,1
3,teaparty.org,fake,(Breitbart) – With his hysterical gotcha attac...,Trump Calls Out Race-Baiting ABC News Reporter,nan,1,1
4,beforeitsnews.com,fake,Researchers develop new depression diagnosis a...,Researchers develop new depression diagnosis a...,Bel Marra Health,0,1
...,...,...,...,...,...,...,...
108006,sports.yahoo.com,reliable,"View photos\nMichigan guard Zak Irvin, right, ...",No. 25 Michigan beats Mount St. Mary's 64-47,The Associated Press,0,0
108007,uk.finance.yahoo.com,reliable,President-elect Donald Trump continued his scr...,"Trump quotes Hillary Clinton, rages against Wi...",Maxwell Tani,0,0
108008,www.yahoo.com,reliable,Holiday shoppers eager to snag big discounts t...,Dialing up deals: Black Friday online sales hi...,ALEX VEIGA,0,0
108009,www.reuters.com,reliable,Kabul police raid shisha cafes in crackdown on...,Kabul police raid shisha cafes in crackdown on...,nan,1,0


In [6]:
%%time
df_news = spark.createDataFrame(df_news_train)
df_news = df_news.withColumn("label", df_news["label"].cast(IntegerType()))
df_news = df_news.withColumn("authors_missing", df_news["authors_missing"].cast(IntegerType()))

CPU times: user 13.1 s, sys: 776 ms, total: 13.8 s
Wall time: 15 s


In [7]:
print(df_news.printSchema())

root
 |-- domain: string (nullable = true)
 |-- type: string (nullable = true)
 |-- content: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_missing: integer (nullable = true)
 |-- label: integer (nullable = true)

None


In [8]:
df_news.show(10)

+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|           domain|type|             content|               title|             authors|authors_missing|label|
+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|beforeitsnews.com|fake|Boehner presses t...|Boehner presses t...|      United Liberty|              0|    1|
|beforeitsnews.com|fake|An Armed Good Sam...|An Armed Good Sam...|       The Real Revo|              0|    1|
|beforeitsnews.com|fake|(Before It's News...|Village Of The De...|Rob Morphy, Mort ...|              0|    1|
|     teaparty.org|fake|(Breitbart) – Wit...|Trump Calls Out R...|                 nan|              1|    1|
|beforeitsnews.com|fake|Researchers devel...|Researchers devel...|    Bel Marra Health|              0|    1|
|beforeitsnews.com|fake|Trading Watch Lis...|Trading Watch Lis...|Bulls On Wall Street|              0|    1|
|beforeits

In [9]:
%%time
# only keep type and content
df_news = df_news.select("label", "content")

#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.content != "")

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)
param_tuning = False

CPU times: user 8.12 ms, sys: 8 µs, total: 8.12 ms
Wall time: 410 ms


In [10]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 106 µs, sys: 9 µs, total: 115 µs
Wall time: 121 µs


In [11]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 110 µs, sys: 0 ns, total: 110 µs
Wall time: 116 µs


In [12]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 16.5 µs


In [13]:
%%time
def run_models(df_train, df_test):
    print("**********LogisticRegression**********")
    t = time.time()
    lr_model = LogisticRegression(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  family='binomial', 
                                  fitIntercept=True, 
                                  threshold=0.5, 
                                  standardization=False, 
                                  maxIter=200, 
                                  regParam=0.005, 
                                  elasticNetParam=0, 
                                  tol=1e-06, 
                                  aggregationDepth=2)

    lr_model = lr_model.fit(df_train)
    
    eval_model_perf(lr_model, df_test)
    
    print("time taken for LogisticRegression: ", time.time() - t)
    t = time.time()

    print("**********DecisionTreeClassifier**********")
    dt_model = DecisionTreeClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction', 
                                      maxDepth=10, maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10,
                                      impurity='gini', 
                                      seed=666)

    dt_model = dt_model.fit(df_train)
    eval_model_perf(dt_model, df_test)
    print("time taken for DecisionTreeClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********RandomForestClassifier**********")
    rf_model = RandomForestClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction',
                                      maxDepth=10, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10, 
                                      impurity='gini', 
                                      numTrees=200, 
                                      featureSubsetStrategy='auto', 
                                      seed=666, 
                                      subsamplingRate=0.8)

    rf_model = rf_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for RandomForestClassifier: ", time.time() - t)
    t = time.time()
    
    print("**********GBTClassifier**********")
    gbt_model = GBTClassifier(featuresCol='features', 
                             labelCol='label', 
                             maxIter=250)

    gbt_model = gbt_model.fit(df_train)
    eval_model_perf(gbt_model, df_test)
    print("time taken for GBTClassifier: ", time.time() - t)
    t = time.time()    
    
    print("**********MultilayerPerceptronClassifier**********")
    mp_model = MultilayerPerceptronClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction', 
                                              layers=[4, 5, 4, 3],  
                                              maxIter=100, 
                                              blockSize=128, 
                                              seed=1234)

    mp_model = mp_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for MultilayerPerceptronClassifier: ", time.time() - t)
    t = time.time()  

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.7 µs


In [14]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["pca_features", "content_features", "word_features"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        VectorAssembler(inputCols=["content_features", "word_features"], outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 11.2 µs


In [ ]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(3000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_pca.take(1))
df_test_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_pca.take(1))
run_models(df_train_pca, df_test_pca)

**********Run Models with PCA Features**********
[Row(label=1, features=DenseVector([-22.0074, 1.7468, 10.5792, 4.9642, 11.3049, -3.0668, -8.332, -1.4447, -15.7509, -1.6806, 2.1436, 1.964, -4.3619, -1.677, -6.9755, -6.0372, 5.4687, -1.7859, -9.5688, 4.0396, 2.7192, -3.603, 3.0286, 5.9583, 0.2917, -8.276, 6.01, -1.843, 2.0768, 1.5102, -0.296, 2.976, 0.2103, 0.6911, 1.2656, 0.3281, 4.053, 3.1602, -2.5984, -1.4353, 4.966, -2.1797, 0.6323, 4.3106, 1.7353, -1.0416, -0.2837, 0.1344, -1.5166, 2.5739, -3.4595, 1.2579, 0.9333, -0.1843, 3.0886, 0.6023, -3.3606, 5.5071, -1.1863, 4.3941, 0.9795, -1.2223, -4.2016, -3.9709, -1.2527, 0.9829, -5.5237, -4.6546, 3.8882, -1.2162, 6.8242, -0.8683, 5.1257, -5.8941, -3.0483, 3.852, -0.6034, 1.4498, 4.4992, -1.7428, 3.627, 0.9651, 4.1241, -1.9163, 3.9631, 3.0485, -3.5137, -1.2811, 4.7492, 3.6088, 0.6963, -0.5498, -2.2639, -2.45, -5.0489, 0.4113, 0.3066, -1.899, -2.7421, -1.5389, 6479.0, 5136.0, 179.0, 0.0349, 618.0, 430.0, 0.6958, 0.0, 0.0, 0.0, 0.0]))]
[Row

In [ ]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(3000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_wo_pca.take(1))
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_wo_pca.take(1))

nb_model = NaiveBayes(featuresCol='features', 
                      labelCol='label', 
                      predictionCol='prediction', 
                      probabilityCol='probability', 
                      rawPredictionCol='rawPrediction', 
                      smoothing=1, 
                      modelType='multinomial')

nb_model = nb_model.fit(df_train_wo_pca)
eval_model_perf(nb_model, df_test_wo_pca)

**********Run Models without PCA Features**********
[Row(label=1, features=DenseVector([6479.0, 5136.0, 179.0, 0.0349, 618.0, 430.0, 0.6958, 0.0, 0.0, 0.0, 0.0]))]
[Row(label=1, features=DenseVector([13886.0, 11027.0, 396.0, 0.0359, 1282.0, 802.0, 0.6256, 0.0, 0.0, 0.0, 0.0]))]
accuracy  =  0.5704556881027469
f1  =  0.5665654197967869
AUC = 0.6240900787267446
CPU times: user 248 ms, sys: 49.5 ms, total: 297 ms
Wall time: 3min 11s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]